In [ ]:
!pip install streamlit tensorflow opencv-python numpy watchdog


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 45.6 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
dataset_path = "/content/drive/My Drive/RC_Dataset"
model_path = "/content/drive/My Drive/rc_verification_model.h5"


In [5]:
!pip install watchdog


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.4 MB/s eta 0:00:00


In [6]:
import os
import time
import tensorflow as tf
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler

# Function to count images in dataset
def count_images():
    original_count = len(os.listdir(f"{dataset_path}/original_rc"))
    photocopy_count = len(os.listdir(f"{dataset_path}/photocopy_rc"))
    return original_count + photocopy_count

# Monitor for new images
class ImageMonitor(FileSystemEventHandler):
    def __init__(self):
        self.last_count = count_images()

    def on_modified(self, event):
        current_count = count_images()
        if current_count > self.last_count:
            print("📢 New images detected! Retraining model...")
            retrain_model()
            self.last_count = current_count

# Function to retrain model
def retrain_model():
    # Reload dataset and retrain
    from tensorflow.keras.preprocessing.image import ImageDataGenerator

    datagen = ImageDataGenerator(
        rescale=1./255, validation_split=0.2
    )

    train_data = datagen.flow_from_directory(
        dataset_path, target_size=(224, 224), batch_size=32, class_mode='binary', subset='training'
    )
    val_data = datagen.flow_from_directory(
        dataset_path, target_size=(224, 224), batch_size=32, class_mode='binary', subset='validation'
    )

    # Load or create a model
    if os.path.exists(model_path):
        model = tf.keras.models.load_model(model_path)
    else:
        model = tf.keras.models.Sequential([
            tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(224, 224, 3)),
            tf.keras.layers.MaxPooling2D(2,2),
            tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
            tf.keras.layers.MaxPooling2D(2,2),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(512, activation='relu'),
            tf.keras.layers.Dropout(0.5),
            tf.keras.layers.Dense(1, activation='sigmoid')
        ])
        model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    # Train model on new images
    model.fit(train_data, validation_data=val_data, epochs=3)
    model.save(model_path)
    print("✅ Model retrained and updated!")

# Start monitoring Google Drive for new images
observer = Observer()
event_handler = ImageMonitor()
observer.schedule(event_handler, dataset_path, recursive=True)
observer.start()

print("🔄 Monitoring for new images in Google Drive...")


🔄 Monitoring for new images in Google Drive...


In [8]:
!pip install streamlit


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 73.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 90.6 MB/s eta 0:00:00


In [9]:
import streamlit as st
import tensorflow as tf
import cv2
import numpy as np
import os
import shutil

# Load trained model
model = tf.keras.models.load_model("/content/drive/My Drive/rc_verification_model.h5")

# Function to process images
def process_image(image_path):
    IMG_SIZE = (224, 224)
    image = cv2.imread(image_path)
    image = cv2.resize(image, IMG_SIZE)
    image = np.expand_dims(image, axis=0) / 255.0
    return image

# Function to predict RC authenticity
def predict_rc(image_path):
    image = process_image(image_path)
    prediction = model.predict(image)[0][0]
    label = "✅ Original RC" if prediction > 0.5 else "❌ Photocopy RC"
    confidence = prediction * 100 if prediction > 0.5 else (1 - prediction) * 100
    return label, round(confidence, 2)

# Streamlit Web UI
st.title("🚗 RC Verification Web App")
st.write("Upload an RC image to check if it's **Original** or a **Photocopy**.")

uploaded_file = st.file_uploader("Upload RC Image", type=["jpg", "jpeg", "png"])

if uploaded_file is not None:
    file_path = "uploaded_rc.jpg"
    with open(file_path, "wb") as f:
        f.write(uploaded_file.getbuffer())

    # Display uploaded image
    st.image(file_path, caption="Uploaded RC Image", use_column_width=True)

    # Get Prediction
    label, confidence = predict_rc(file_path)

    # Show Prediction Result
    st.subheader("🔍 Prediction Result")
    st.write(f"**Result:** {label}")
    st.write(f"**Confidence Score:** {confidence}%")

    # Allow user to correct wrong predictions
    if st.button("Move to Correct Folder"):
        correct_folder = "original_rc" if "Original" in label else "photocopy_rc"
        correct_path = f"/content/drive/My Drive/RC_Dataset/{correct_folder}/{uploaded_file.name}"
        shutil.move(file_path, correct_path)
        st.write(f"✅ Image moved to `{correct_folder}`! Model will retrain automatically.")


2025-02-28 06:55:47.187 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-28 06:55:47.383 
  command:

    streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-02-28 06:55:47.384 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-28 06:55:47.386 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-28 06:55:47.387 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-28 06:55:47.389 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-28 06:55:47.390 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-28 06:55:47.391 Thread 'MainThread': mi

In [10]:
!pip install streamlit pyngrok


In [12]:
%%writefile app.py
import streamlit as st
import tensorflow as tf
import cv2
import numpy as np
import os

# Load trained model
model = tf.keras.models.load_model("rc_verification_model.h5")

# Function to process images
def process_image(image_path):
    IMG_SIZE = (224, 224)
    image = cv2.imread(image_path)
    image = cv2.resize(image, IMG_SIZE)
    image = np.expand_dims(image, axis=0) / 255.0
    return image

# Function to predict RC authenticity
def predict_rc(image_path):
    image = process_image(image_path)
    prediction = model.predict(image)[0][0]
    label = "✅ Original RC" if prediction > 0.5 else "❌ Photocopy RC"
    confidence = prediction * 100 if prediction > 0.5 else (1 - prediction) * 100
    return label, round(confidence, 2)

# Streamlit Web UI
st.title("🚗 RC Verification Web App")
st.write("Upload an RC image to check if it's **Original** or a **Photocopy**.")

uploaded_file = st.file_uploader("Upload RC Image", type=["jpg", "jpeg", "png"])

if uploaded_file is not None:
    file_path = "uploaded_rc.jpg"
    with open(file_path, "wb") as f:
        f.write(uploaded_file.getbuffer())

    # Display uploaded image
    st.image(file_path, caption="Uploaded RC Image", use_column_width=True)

    # Get Prediction
    label, confidence = predict_rc(file_path)

    # Show Prediction Result
    st.subheader("🔍 Prediction Result")
    st.write(f"**Result:** {label}")
    st.write(f"**Confidence Score:** {confidence}%")


Writing app.py


In [13]:
!ngrok config add-authtoken 2tczXl4E9OHYqpsRJFAtXr45TDa_4R13nYDmkhvoybKoyRmh2

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from pyngrok import ngrok

# Run Streamlit in background
!streamlit run app.py &

# Expose the Streamlit port (8501) via ngrok
public_url = ngrok.connect(port='8501')
print(f"🔗 Click here to open your Streamlit App: {public_url}")





  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.105.43.180:8501



In [ ]:
%%writefile app.py
import streamlit as st
import tensorflow as tf
import cv2
import numpy as np
import os
import shutil

# Load trained model
model = tf.keras.models.load_model("/content/drive/My Drive/rc_verification_model.h5")

# Function to process images
def process_image(image_path):
    IMG_SIZE = (224, 224)
    image = cv2.imread(image_path)
    image = cv2.resize(image, IMG_SIZE)
    image = np.expand_dims(image, axis=0) / 255.0
    return image

# Function to predict RC authenticity
def predict_rc(image_path):
    image = process_image(image_path)
    prediction = model.predict(image)[0][0]
    label = "✅ Original RC" if prediction > 0.5 else "❌ Photocopy RC"
    confidence = prediction * 100 if prediction > 0.5 else (1 - prediction) * 100
    return label, round(confidence, 2)

# Streamlit Web UI
st.title("🚗 RC Verification Web App")
st.write("Upload an RC image to check if it's **Original** or a **Photocopy**.")

uploaded_file = st.file_uploader("Upload RC Image", type=["jpg", "jpeg", "png"])

if uploaded_file is not None:
    file_path = "uploaded_rc.jpg"
    with open(file_path, "wb") as f:
        f.write(uploaded_file.getbuffer())

    # Display uploaded image
    st.image(file_path, caption="Uploaded RC Image", use_column_width=True)

    # Get Prediction
    label, confidence = predict_rc(file_path)

    # Show Prediction Result
    st.subheader("🔍 Prediction Result")
    st.write(f"**Result:** {label}")
    st.write(f"**Confidence Score:** {con


Writing app.py


In [ ]:
!ngrok config add-authtoken YOUR_NGROK_AUTHTOKEN


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from pyngrok import ngrok

# Run Streamlit in background
!streamlit run app.py &

# Expose the Streamlit port (8501) via ngrok
public_url = ngrok.connect(port='8501')
print(f"🔗 Click here to open your Streamlit App: {public_url}")





  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.106.187.238:8501

  Stopping...


ERROR:pyngrok.process.ngrok:t=2025-02-27T14:24:06+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: The authtoken you specified is an ngrok v1 authtoken, but you're using ngrok v2.\nYour authtoken: YOUR_NGROK_AUTHTOKEN\nInstructions to install your authtoken are on your ngrok dashboard:\nhttps://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_106\r\n"
ERROR:pyngrok.process.ngrok:t=2025-02-27T14:24:06+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: The authtoken you specified is an ngrok v1 authtoken, but you're using ngrok v2.\nYour authtoken: YOUR_NGROK_AUTHTOKEN\nInstructions to install your authtoken are on your ngrok dashboard:\nhttps://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_106\r\n"
ERROR:pyngrok.process.ngrok:t=2025-02-27T14:24:06+0000 lvl=eror msg="terminating with error" obj=app err="authentication failed: The authtoken you specified is an ngrok v1 authtoke

PyngrokNgrokError: The ngrok process errored on start: authentication failed: The authtoken you specified is an ngrok v1 authtoken, but you're using ngrok v2.\nYour authtoken: YOUR_NGROK_AUTHTOKEN\nInstructions to install your authtoken are on your ngrok dashboard:\nhttps://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_106\r\n.

In [ ]:
!pip install pyngrok


In [ ]:
ngrok config add-authtoken 2tczXl4E9OHYqpsRJFAtXr45TDa_4R13nYDmkhvoybKoyRmh2

SyntaxError: invalid decimal literal (<ipython-input-13-1deb126015dc>, line 1)

In [ ]:
!ngrok config add-authtoken 2tczXl4E9OHYqpsRJFAtXr45TDa_4R13nYDmkhvoybKoyRmh2

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from pyngrok import ngrok

# Run Streamlit in background
!streamlit run app.py &

# Expose the Streamlit port (8501) via ngrok
public_url = ngrok.connect(port='8501')
print(f"🔗 Click here to open your Streamlit App: {public_url}")





  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.106.187.238:8501



In [1]:
!ls "/content/drive/My Drive/RC_Dataset/"



ls: cannot access '/content/drive/My Drive/RC_Dataset/': No such file or directory
